In [213]:
import pandas as pd
import PyPDF2, re

def getTableOfContents(pdf_path):
    pdfReader = PyPDF2.PdfReader(open(pdf_path, 'rb'))

    title_and_numPage = []
    df = pd.DataFrame(columns=['title', 'numPage', 'nextPage'])


    for numPage in range(9,88):
        # print("Current page: ", numPage + 1)
        text = pdfReader.pages[numPage].extract_text()
        regex = r"([0-9]+\.+([0-9]\.)*([0-9]* | ))"

        for title in re.findall(regex, text):

            if len(title[0].split('.')[0]) > 2:
                print("Ignore years")
                continue # To ignore cases like "2020."

            regex_to_check_invalid_titles = r"([a-zA-Z,;:] |[a-zA-Z-])"+title[0]

            if len(re.findall(regex_to_check_invalid_titles, text)) > 0:
                print("Invalid title: ", title[0])
                continue


            title_and_numPage.append((title[0], numPage + 1))

            previousTitle = len(title_and_numPage)-2

            if previousTitle < 0 : continue


            pattern = r"\d+<" #to remove the page number indicator (example "35<")
            text = re.sub(pattern," ",text)
            # print(text)
            index = text.find(title[0])

            if index > 0 and any(c != ' ' for c in text[:index]): # if the title is not the first word of the page
                df.loc[len(df)] = [
                    title_and_numPage[previousTitle][0],
                    title_and_numPage[previousTitle][1],
                    numPage + 1
                ]
            else:
                df.loc[len(df)] = [
                    title_and_numPage[previousTitle][0],
                    title_and_numPage[previousTitle][1],
                    numPage # the previous page
                ]

            if numPage == 87: # last page
                df.loc[len(df)] = [
                    title[0],
                    numPage + 1,
                    numPage + 1
                ]

    df['title'] = df['title'].apply(lambda x: x.replace('.', '_'))

    return df



In [215]:
path = "Catalogue European.pdf"

result = getTableOfContents(path)

Invalid title:  2.5 
Ignore years
Ignore years
Ignore years
Invalid title:  1. 
Invalid title:  2. 
Invalid title:  3. 
Ignore years
Ignore years
Invalid title:  19. 
Ignore years
Invalid title:  2. 
Ignore years
Invalid title:  10.000 
Ignore years
Ignore years


In [216]:
result

,title,numPage,nextPage
0,4_,10,10
1,4_1,10,10
2,4_1_1,10,10
3,4_1_1_1,10,10
4,4_1_1_2,11,12
...,...,...,...
85,4_4_3_11,86,86
86,4_4_3_12,86,86
87,4_4_3_13,87,87
88,4_4_3_14,87,87


In [218]:
result.to_csv("newUE.csv")